In [61]:
%load_ext zipline
%matplotlib inline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [154]:
from zipline.utils import factory
from zipline.api import symbol, order, record
import numpy as np
from datetime import datetime
import pytz
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
import os
import collections
from StringIO import StringIO

dictData = collections.OrderedDict()

parse = lambda x: datetime.strptime(x, '%d-%m-%y').date()

dictData['DOGEBTC'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/DOGEBTC.csv', index_col=['Timestamp'], 
                                  usecols=['Timestamp','Open','High','Low','Close','Volume'], sep=';',
                                  parse_dates=True, date_parser=parse)
# dictData['MAIDBTC'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/MAIDBTC.csv', index_col=['Timestamp'], usecols=['Timestamp','Open','High','Low','Close','Volume','VWAP','TWAP'], sep=';')
# dictData['DASHBTC'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/DASHBTC.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['LTCUSD'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/LTCUSD.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['XRPUSD'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/XRPUSD.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['BTCETH'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/BTCETH.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['BTSBTC'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/BTSBTC.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['XMRBTC'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/XMRBTC.csv', parse_dates=True, index_col=False, date_parser=parse, sep=';')
# dictData['BTCUSD'] = pd.read_csv('/Users/JH/Desktop/Business/GGSB Thesis 2016/Code/excelled data/BTCUSD.csv', index_col=['Timestamp'], usecols=['Timestamp','Open','High','Low','Close','Volume','VWAP','TWAP'], sep=';')
print(dictData['DOGEBTC'])

                Open      High       Low     Close         Volume
Timestamp                                                        
2014-04-01  0.000570  0.000584  0.000558  0.000565   264147.21430
2014-04-02  0.000562  0.000575  0.000461  0.000468   334533.40500
2014-04-03  0.000468  0.000483  0.000410  0.000483   439332.31670
2014-04-04  0.000483  0.000504  0.000342  0.000482   285268.15880
2014-04-05  0.000483  0.000485  0.000068  0.000481   252173.91660
2014-04-06  0.000481  0.000491  0.000455  0.000468   208399.63040
2014-04-07  0.000470  0.000482  0.000456  0.000475   363096.95550
2014-04-08  0.000471  0.000677  0.000446  0.000459   280283.02690
2014-04-09  0.000461  0.000474  0.000441  0.000454   258227.76170
2014-04-10  0.000455  0.000455  0.000082  0.000337   343927.03090
2014-04-11  0.000334  0.000441  0.000328  0.000441   404350.44250
2014-04-12  0.000441  0.000451  0.000414  0.000421   199680.84110
2014-04-13  0.000421  0.000428  0.000365  0.000385   272674.26960
2014-04-14

In [158]:
panel = pd.Panel(dictData)
panel.minor_axis = ['open', 'high', 'low', 'close', 'volume']
print(panel)
print(panel.minor_axis)
print(panel['DOGEBTC'])

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 803 (major_axis) x 5 (minor_axis)
Items axis: DOGEBTC to DOGEBTC
Major_axis axis: 2014-04-01 00:00:00 to 2016-06-11 00:00:00
Minor_axis axis: open to volume
Index([u'open', u'high', u'low', u'close', u'volume'], dtype='object')
                open      high       low     close         volume
Timestamp                                                        
2014-04-01  0.000570  0.000584  0.000558  0.000565   264147.21430
2014-04-02  0.000562  0.000575  0.000461  0.000468   334533.40500
2014-04-03  0.000468  0.000483  0.000410  0.000483   439332.31670
2014-04-04  0.000483  0.000504  0.000342  0.000482   285268.15880
2014-04-05  0.000483  0.000485  0.000068  0.000481   252173.91660
2014-04-06  0.000481  0.000491  0.000455  0.000468   208399.63040
2014-04-07  0.000470  0.000482  0.000456  0.000475   363096.95550
2014-04-08  0.000471  0.000677  0.000446  0.000459   280283.02690
2014-04-09  0.000461  0.000474  0.000441  0.000454   2

In [159]:
sim_params = factory.create_simulation_parameters(
    start=pd.to_datetime("01-04-14", dayfirst=True).tz_localize("Europe/London").tz_convert("UTC"),
    end=pd.to_datetime("11-06-16", dayfirst=True).tz_localize("Europe/London").tz_convert("UTC"),
    data_frequency="daily",
    emission_rate="daily")

#data = data.dropna()

In [161]:
from zipline.algorithm import TradingAlgorithm
from zipline.api import order, symbol

import matplotlib.pyplot as plt

def initialize(algo):
    pass

# This will be the ETF trading strategy
def handle_data(algo, data):
    order(symbol('DOGEBTC'), 10)


algo_obj = TradingAlgorithm(initialize=initialize, handle_data=handle_data, sim_params=sim_params)
perf_manual = algo_obj.run(data=panel, overwrite_sim_params=False)


def analyze(algo, perf_manual):
    ax1 = plt.subplot(211)
    perf_manual.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf_manual.DOGEBTC.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()